<a href="https://colab.research.google.com/github/IM07813/Transformers/blob/main/essay_eval__.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install transformers




Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
# Import libraries
import tensorflow as tf
import transformers
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer

# Load pre-trained model and tokenizer
model_name = "bert-base-cased"
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6) # Assuming 6-point scale for essay scores
tokenizer = AutoTokenizer.from_pretrained(model_name)
print(transformers.__version__)
print(tf.__version__)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


4.29.2
2.12.0
